In [1]:
import requests
import json
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 開高低收 #

In [2]:
def get_today_price(stock, date):
    url='http://www.twse.com.tw/exchangeReport/STOCK_DAY'
    params = {}
    params['stockNo'] = stock
    params['date'] = date
    params['response'] = json
    res = requests.get(url, params=params)
    s = json.loads(res.text)
    rawData=pd.DataFrame(s['data'],columns=['日期', '成交股數', '成交金額', '開盤價','最高價','最低價','收盤價','漲跌價差','成交筆數'])
    return rawData

In [3]:
get_today_price(2330, 20180106)[-1:]

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
5,107/01/09,"19,043,123","4,588,314,012",242.00,242.00,239.50,242.00,0.00,"7,124"


# 籌碼面

In [4]:
def get_Chips(stocktype, date):
    url = 'http://www.twse.com.tw/fund/T86'
    params = {}
    params['selectType'] = stocktype
    params['date'] = date
    params['response'] = json

    res = requests.get(url, params=params)
    s = json.loads(res.text)
    rawData=pd.DataFrame(s['data'],columns=['證券代號','證券名稱','外陸資買進股數(不含外資自營商)','外陸資賣出股數(不含外資自營商)',
                                     '外陸資買賣超股數(不含外資自營商)','外資自營商買進股數','外資自營商賣出股數','外資自營商買賣超股數',
                                     '投信買進股數','投信賣出股數','投信買賣超股數','自營商買賣超股數','自營商買進股數(自行買賣)','自營商賣出股數(自行買賣)',
                                     '自營商買賣超股數(自行買賣)','自營商買進股數(避險)','自營商賣出股數(避險)','自營商買賣超股數(避險)','三大法人買賣超股數'])
    return rawData

In [5]:
d=get_Chips(24,20180105)
d=d[d['證券代號']=='2330']
d.iloc[:,[0,1,4,10,11,18]]

,證券代號,證券名稱,外陸資買賣超股數(不含外資自營商),投信買賣超股數,自營商買賣超股數,三大法人買賣超股數
0,2330,台積電,"9,298,675","23,000","-12,000","9,309,675"


# 資券比 

In [6]:
def get_lend(stocktype, date):
    url = 'http://www.twse.com.tw/exchangeReport/MI_MARGN'
    params = {}
    params['selectType'] = stocktype
    params['date'] = date
    params['response'] = json

    res = requests.get(url, params=params)
    s = json.loads(res.text)
    rawData=pd.DataFrame(s['data'],columns=s['fields'])
    return rawData

In [7]:
get_lend(24,20180105).head()

,股票代號,股票名稱,買進,賣出,現金償還,前日餘額,今日餘額,限額,買進,賣出,現金償還,前日餘額,今日餘額,限額,資券互抵,註記
0,1437,勤益控,35,7,0,"2,337","2,365","48,421",0,0,0,0,0,"48,421",0,
1,2302,麗正,0,5,0,"3,257","3,252",0,0,0,0,0,0,0,0,OX
2,2303,聯電,715,"2,046",10,"76,281","74,940","3,156,079",14,122,0,"5,603","5,711","3,156,079",21,
3,2311,日月光,128,719,0,"19,425","18,834","2,183,071",5,27,0,209,231,"2,183,071",0,
4,2325,矽品,0,2,0,221,219,"779,090",0,0,0,17,17,"779,090",0,


# 歷史資料計算技術指標

In [5]:
import pandas as pd
df = pd.read_csv('tsmc.csv',header=None,sep=',')
df.columns=['date','open','high','low','close','volume','long','short','foreign','invest','self','total']
df.head()

,date,open,high,low,close,volume,long,short,foreign,invest,self,total
0,2008/01/02,41.02,41.02,39.83,40.03,38970.0,50951.0,1894.0,394.0,-8824.0,-365.0,-8795.0
1,2008/01/03,39.63,39.70,39.30,39.30,34838.0,52326.0,1615.0,-9307.0,-804.0,482.0,-9629.0
2,2008/01/04,39.24,39.96,38.71,39.70,48400.0,52437.0,1533.0,-6802.0,-1357.0,711.0,-7448.0
3,2008/01/07,37.71,38.38,36.92,36.92,103229.0,60706.0,2510.0,-65640.0,-5021.0,-1424.0,-72085.0
4,2008/01/08,36.92,37.45,36.72,37.05,47333.0,60340.0,2520.0,-13831.0,77.0,-1248.0,-15002.0


In [6]:
#技術指標
import pandas as pd
df = pd.read_csv('tsmc.csv',header=None,sep=',')
df.columns=['date','open','high','low','close','volume','long','short','foreign','invest','self','total']
tsmc = {
    'close':df.close.dropna().astype(float),
    'open':df.open.dropna().astype(float),
    'high':df.high.dropna().astype(float),
    'low':df.low.dropna().astype(float),
    'volume': df.volume.dropna().astype(float),
    'long': df.long.dropna().astype(float),
    'short': df.short.dropna().astype(float),
    'foreign': df.foreign.dropna().astype(float),
    'invest': df.invest.dropna().astype(float),
    'self': df.self.dropna().astype(float),
    'total': df.total.dropna().astype(float),
}


import talib
def talib2df(talib_output):
    if type(talib_output) == list:
        ret = pd.DataFrame(talib_output).transpose()
    else:
        ret = pd.Series(talib_output)
    ret.index = df['close'].index
    return ret;
#計算KD#
KD = talib2df(talib.abstract.STOCH(tsmc, fastk_period=9))
#計算MACD#
MACD = talib2df(talib.abstract.MACD(tsmc))
#計算OBV#
OBV = talib2df(talib.abstract.OBV(tsmc))
#計算威廉指數#
WILLR = talib2df(talib.abstract.WILLR(tsmc))
#ATR 計算#
ATR = talib2df(talib.abstract.ATR(tsmc))
#alldata
tsmc=pd.DataFrame(tsmc)
tsmc = pd.concat([df,KD,MACD,OBV,WILLR,ATR], axis=1)
#tsmc.tail()
tsmc.columns=['date','open','high','low','close','volume','long','short','foreign','invest','self','total','k','d','dif12','dif26','macd','obv','willr','atr']
#tsmc.tail()

# 機器學習預測力

In [9]:
import numpy as np
import pandas as pd
from sklearn import  model_selection,ensemble, preprocessing, metrics

# 移除遺漏值、與設漲即為1
df=tsmc
df['label']=(df.close - df.close.shift(1)) > 0
df=df.dropna()

# 創造 dummy variables
label_encoder = preprocessing.LabelEncoder()
encoded_label = label_encoder.fit_transform(df["label"])

# 建立訓練與測試資料
tsmc_X = df[['open','high','low','close','volume','long','short','foreign','invest','self','total','k','d','macd','obv','willr','atr']]
tsmc_y = df['label']
train_X, test_X, train_y, test_y = model_selection.train_test_split(tsmc_X, tsmc_y, test_size = 0.3)

# 建立 random forest 模型
forest = ensemble.RandomForestClassifier(n_estimators = 100)
forest_fit = forest.fit(train_X, train_y)

# 預測
test_y_predicted = forest.predict(test_X)
#test_y_predicted
# 績效
accuracy = metrics.accuracy_score(test_y, test_y_predicted)
#print(accuracy)
fpr, tpr, thresholds = metrics.roc_curve(test_y, test_y_predicted)
auc = metrics.auc(fpr, tpr)
#print(auc)
print('準確率: {}'.format(auc))
print('AUC值: {}'.format(accuracy))

準確率: 0.7873239436619718
AUC值: 0.7877551020408163


# 合併今日資料計算技術指標

In [ ]:
20180108,(20180110-20180108+1)+20180108

In [82]:
def getTWDate(dt):
    d =str(dt)
    year      = int(d[0:4]) -1911
    monthdate = d[4:6]
    date = d[6:8]
    ymd = '{}/{}/{}'.format(year, monthdate,date)
    return ymd

In [81]:
k=get_today_price(2330, 20180109)
ymd=getTWDate(20180109)
k=k[k['日期']==ymd]
#k.index=[0]
k

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數


In [83]:
df = pd.read_csv('tsmc.csv',header=None,sep=',')
df.columns=['date','open','high','low','close','volume','long','short','foreign','invest','self','total']
for i in range(20180102,(20180105-20180102+1)+20180102):
    #整合資訊
    k=get_today_price(2330, i)
    ymd=getTWDate(i)
    k=k[k['日期']==ymd]
    k.index=[0]
    #買賣超
    a=get_Chips(24,i)
    a=a[a['證券代號']=='2330']
    a=a.iloc[:,[0,1,4,10,11,18]]
    a.index=[0]
    #資券餘額
    b=get_lend(24,i)
    b=b[b['股票代號']=='2330']
    b=b.iloc[:,[6,12]]
    b.index=[0]
    df1 = pd.concat([k,a,b], axis=1)
    cols = df1.columns.tolist()
    cols=cols[:1]+cols[3:7]+cols[1:2]+cols[15:16]+cols[11:15]
    df1=df1[cols]
    df1.columns=['date','open','high','low','close','volume','long','short','foreign','invest','self','total']
    df1.volume=''.join(df1.volume.values).replace(',','')
    df1.long=''.join(df1.long.values).replace(',','')
    df1.short=''.join(df1.short.values).replace(',','')
    df1.foreign=''.join(df1.foreign.values).replace(',','')
    df1.invest=''.join(df1.invest.values).replace(',','')
    df1.self=''.join(df1.self.values).replace(',','')
    df1.total=''.join(df1.total.values).replace(',','')
    df1[['open','high','low','close','volume','long','short','foreign','invest','self','total']] = df1[['open','high','low','close','volume','long','short','foreign','invest','self','total']].apply(pd.to_numeric)
    df1[['volume','foreign','invest','self','total']] = df1[['volume','foreign','invest','self','total']].apply(lambda x: round(x/1000))
    #合併舊資料做kd
    df=df.append(df1)

In [84]:
df.tail()

,date,open,high,low,close,volume,long,short,foreign,invest,self,total
2479,2018/01/05,240.0,240.0,238.0,240.0,23721.0,8554.0,2526.0,9299.0,23.0,-12.0,9310.0
0,107/01/02,231.5,232.5,231.0,232.5,18055.0,9703.0,2161.0,9607.0,514.0,53.0,10174.0
0,107/01/03,236.0,238.0,235.5,237.0,31706.0,9434.0,2321.0,8839.0,915.0,467.0,10221.0
0,107/01/04,240.0,240.0,236.5,239.5,29180.0,9066.0,2395.0,10930.0,47.0,-30.0,10947.0
0,107/01/05,240.0,240.0,238.0,240.0,23721.0,8554.0,2526.0,9299.0,23.0,-12.0,9310.0


In [11]:
#將今日資料與之前合併算出技術指標值 預測今日
#整合資訊
k=get_today_price(2330, 20180105)[-1:]
k.index=[0]
#買賣超
a=get_Chips(24,20180105)
a=a[a['證券代號']=='2330']
a=a.iloc[:,[0,1,4,10,11,18]]
a.index=[0]
#資券餘額
b=get_lend(24,20180105)
b=b[b['股票代號']=='2330']
b=b.iloc[:,[6,12]]
b.index=[0]

df1 = pd.concat([k,a,b], axis=1)
cols = df1.columns.tolist()
cols=cols[:1]+cols[3:7]+cols[1:2]+cols[15:16]+cols[11:15]
df1=df1[cols]
df1.columns=['date','open','high','low','close','volume','long','short','foreign','invest','self','total']
df1.volume=''.join(df1.volume.values).replace(',','')
df1.long=''.join(df1.long.values).replace(',','')
df1.short=''.join(df1.short.values).replace(',','')
df1.foreign=''.join(df1.foreign.values).replace(',','')
df1.invest=''.join(df1.invest.values).replace(',','')
df1.self=''.join(df1.self.values).replace(',','')
df1.total=''.join(df1.total.values).replace(',','')
df1[['open','high','low','close','volume','long','short','foreign','invest','self','total']] = df1[['open','high','low','close','volume','long','short','foreign','invest','self','total']].apply(pd.to_numeric)
df1[['volume','foreign','invest','self','total']] = df1[['volume','foreign','invest','self','total']].apply(lambda x: round(x/1000))
#合併舊資料做kd
df = pd.read_csv('tsmc.csv',header=None,sep=',')
df.columns=['date','open','high','low','close','volume','long','short','foreign','invest','self','total']
df=df.append(df1)
tsmc = {
    'close':df.close.dropna().astype(float),
    'open':df.open.dropna().astype(float),
    'high':df.high.dropna().astype(float),
    'low':df.low.dropna().astype(float),
    'volume': df.volume.dropna().astype(float),
    'long': df.long.dropna().astype(float),
    'short': df.short.dropna().astype(float),
    'foreign': df.foreign.dropna().astype(float),
    'invest': df.invest.dropna().astype(float),
    'self': df.self.dropna().astype(float),
    'total': df.total.dropna().astype(float),
}
import talib
def talib2df(talib_output):
    if type(talib_output) == list:
        ret = pd.DataFrame(talib_output).transpose()
    else:
        ret = pd.Series(talib_output)
    ret.index = df['close'].index
    return ret;
#計算KD#
KD = talib2df(talib.abstract.STOCH(tsmc, fastk_period=9))
#計算MACD#
MACD = talib2df(talib.abstract.MACD(tsmc))
#計算OBV#
OBV = talib2df(talib.abstract.OBV(tsmc))
#計算威廉指數#
WILLR = talib2df(talib.abstract.WILLR(tsmc))
#ATR 計算#
ATR = talib2df(talib.abstract.ATR(tsmc))
#alldata
tsmc=pd.DataFrame(tsmc)
tsmc = pd.concat([df,KD,MACD,OBV,WILLR,ATR], axis=1)
#tsmc.tail()
tsmc.columns=['date','open','high','low','close','volume','long','short','foreign','invest','self','total','k','d','dif12','dif26','macd','obv','willr','atr']
tsmc['label']=(tsmc.close - tsmc.close.shift(1)) > 0
tsmc=tsmc.iloc[-1:]

# 利用之前的模型做預測明日漲跌幅

In [12]:
import numpy as np
#今日資料
today_X = tsmc[['open','high','low','close','volume','long','short','foreign','invest','self','total','k','d','macd','obv','willr','atr']]
# 預測
today_y_predicted = forest.predict(today_X)
proba = forest.predict_proba(today_X)
print('明日走勢: {}'.format(np.where(today_y_predicted==True,'漲','跌')[0]))
print( '明日漲的機率: {}'.format(proba[0][1]))

明日走勢: 漲
明日漲的機率: 0.95
